In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
from pyspark.sql import functions as F
import os
import shutil

def spark_doubling(input, output):
    # TODO: spark.executor.memory and spark.driver.memory
    dfs_best_queue = []
    spark = (
        SparkSession.builder.master("spark://master:7077")
        .appName("Spark doubling")
        .config("spark.executor.memory", "2g")
        .config("spark.driver.memory", "3g")
        .getOrCreate()
    )
    os.mkdir('_checkpoints')
    spark.sparkContext.setCheckpointDir('_checkpoints')
    df = spark.read.csv(input, header=True, inferSchema=True)

    assert df.columns == ["edge_1", "edge_2", "length"], f"got {df.columns}"

    # Aggregate min edge values
    df_best = df.groupBy(["edge_1", "edge_2"]).agg(F.min("length").alias("length"))
    dfs_best_queue.append(df_best.cache())
    dfs_best_queue.append(dfs_best_queue[-1].checkpoint())
    
    cur_paths, sum_cur_length = -1, -1
    iter = 0
    while True:
        iter += 1
        joined_df = (
            dfs_best_queue[-1].alias("a")
            .join(
                dfs_best_queue[-1].alias("b"),
                F.col("a.edge_2") == F.col("b.edge_1"),
                "inner",
            )
            .select(
                F.col("a.edge_1").alias("edge_1"),
                F.col("b.edge_2").alias("edge_2"),
                (F.col("a.length") + F.col("b.length")).alias("length"),
            )
        )
        # Update the main DataFrame
        # TODO: test join method
        df_best = dfs_best_queue[-1].union(joined_df)
        df_best = df_best.groupBy(["edge_1", "edge_2"]).agg(
            F.min("length").alias("length")
        )
        dfs_best_queue.append(df_best.cache())
        
        rm_checkpoints = []
        if iter % 6 == 0:
            for tmp in os.listdir("_checkpoints"):
                for dir in os.listdir(os.path.join("_checkpoints", tmp)):
                    rm_checkpoints.append(os.path.join("_checkpoints", tmp, dir))

        if iter % 3 == 0:
            dfs_best_queue.append(dfs_best_queue[-1].checkpoint())
            for i in range(len(dfs_best_queue)-1):
                dfs_best_queue[i].unpersist()
            dfs_best_queue = [dfs_best_queue[-1]]
        
        for ckpt in rm_checkpoints:
            shutil.rmtree(ckpt)
        
        
        print(f"[debug] cur_paths: {cur_paths}, sum cur length: {sum_cur_length}")
        print(
            f"[debug] {iter}, len of caches dfs {len(spark.sparkContext._jsc.getPersistentRDDs().items())}"
        )

        df_best.printSchema()

        next_cur_paths = dfs_best_queue[-1].count()
        next_sum_cur_length = dfs_best_queue[-1].select(
            sum("length").alias("sum_length")
        ).collect()[0]["sum_length"]

        if (next_cur_paths, next_sum_cur_length) == (cur_paths, sum_cur_length):
            break
            
        cur_paths, sum_cur_length = next_cur_paths, next_sum_cur_length

    # Write result to a single CSV file
    dfs_best_queue[-1].toPandas().to_csv(output, header=True, index=False)
    spark.stop()
    shutil.rmtree("_checkpoints")

In [3]:
spark_doubling("bmlhw1/data/facebook_combined.csv", "trash.csv")

23/12/14 18:08:57 WARN SparkContext: Spark is not running in local mode, therefore the checkpoint directory must not be on the local filesystem. Directory '_checkpoints' appears to be on the local filesystem.
23/12/14 18:09:07 WARN TaskSetManager: Lost task 0.0 in stage 0.0 (TID 0) (10.186.0.9 executor 0): org.apache.spark.SparkFileNotFoundException: File file:/home/jo418361/bmlhw1/data/facebook_combined.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileS

Py4JJavaError: An error occurred while calling o35.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (10.186.0.9 executor 0): org.apache.spark.SparkFileNotFoundException: File file:/home/jo418361/bmlhw1/data/facebook_combined.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.F

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
from pyspark.sql import functions as F
import os
import shutil
spark = (
        SparkSession.builder.master("local[*]")
        .appName("Spark doubling")
        .config("spark.executor.memory", "2g")
        .config("spark.driver.memory", "3g")
        .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/14 18:19:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv("bmlhw1/data/facebook_combined.csv", header=True, inferSchema=True)

In [8]:
df.write.option("header", "true").csv("hdfs://master:9000/facebook.csv")

23/12/14 18:55:16 WARN DataStreamer: DataStreamer Exception         (0 + 1) / 1]
org.apache.hadoop.ipc.RemoteException(java.io.IOException): File /facebook.csv/_temporary/0/_temporary/attempt_20231214185515759916540393481501_0002_m_000000_2/part-00000-8ed07d7b-fd54-4b90-a372-54456b8f35d2-c000.csv could only be written to 0 of the 1 minReplication nodes. There are 0 datanode(s) running and 0 node(s) are excluded in this operation.
	at org.apache.hadoop.hdfs.server.blockmanagement.BlockManager.chooseTarget4NewBlock(BlockManager.java:2350)
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.chooseTargetForNewBlock(FSDirWriteFileOp.java:294)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getAdditionalBlock(FSNamesystem.java:2989)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.addBlock(NameNodeRpcServer.java:912)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.addBlock(ClientNamenodeProtocolServerSideTranslatorPB.java:595)
	a

Py4JJavaError: An error occurred while calling o62.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (driver.europe-central2-a.c.sonic-terrain-380002.internal executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to hdfs://master:9000/facebook.csv.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:774)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:890)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.